In [1]:
import openpyxl

In [2]:
import pandas as pd

In [3]:
path = '../Data/raw/ADAPTIVETransactionSearchPLJeremyResults236.xls'

In [4]:
from xml.sax import ContentHandler, parse

# Reference https://goo.gl/KaOBG3
class ExcelHandler(ContentHandler):
    def __init__(self):
        self.chars = [  ]
        self.cells = [  ]
        self.rows = [  ]
        self.tables = [  ]
    def characters(self, content):
        self.chars.append(content)
    def startElement(self, name, atts):
        if name=="Cell":
            self.chars = [  ]
        elif name=="Row":
            self.cells=[  ]
        elif name=="Table":
            self.rows = [  ]
    def endElement(self, name):
        if name=="Cell":
            self.cells.append(''.join(self.chars))
        elif name=="Row":
            self.rows.append(self.cells)
        elif name=="Table":
            self.tables.append(self.rows)

excelHandler = ExcelHandler()
parse('../Data/raw/ADAPTIVETransactionSearchPLJeremyResults236.xls', excelHandler)
df1 = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])

In [5]:
data1 = df1.to_dict('records')
#This gives us a list of dictionaries.


In [6]:
len(data1)

85777

In [7]:
data1[0]

{'Date': '2020-07-01T00:00:00',
 'Period': 'Jul 2020',
 'GL Account': '5010 Contributions Revenue : Foundation Contributions',
 'Document Number': 'JE5152',
 'Program': 'Development',
 'Initiative': 'DEV - Admin',
 'Project': 'Donation : Donation',
 'Restriction': '2-Temporary Restricted',
 'Grant': 'WRI - Islands FY20',
 'Amount': '50000.00',
 'Vendor Name': '',
 'Employee Name': '',
 'Memo': 'WRI - Islands FY20',
 'Amount (Credit)': '50000.00',
 'Amount (Debit)': '',
 'Client': '',
 'Transaction Number': 'JOURNAL5152',
 'Type': 'Journal',
 'Vendor Rate': '',
 'Vendor Contract Expiration Date': ''}

In [8]:
#Keys are program names, and values are lists of all rows in that program.

programs = {}

for row in data1:
    row[row['Period']] = float(row['Amount'])
    row['Total'] = float(row['Amount'])
    filtered_list = programs.get(row["Program"], [])
    filtered_list.append(row)
    programs[row["Program"]] = filtered_list

    


In [9]:
programs['Climate Aligned Industries']

[{'Date': '2020-11-01T00:00:00',
  'Period': 'Nov 2020',
  'GL Account': '7210 Consultants and Contractors : Consultant Fees',
  'Document Number': '2020-004',
  'Program': 'Climate Aligned Industries',
  'Initiative': 'CAI - Center for Climate Aligned Finance',
  'Project': '*Restricted : Center for Climate Aligned Finance',
  'Restriction': '1-Unrestricted',
  'Grant': 'Wells Fargo - GCF CCAF FY20',
  'Amount': '2025.00',
  'Vendor Name': 'Mitchell, James',
  'Employee Name': '',
  'Memo': 'James Mitchel - Inv #: 2020-004, 9/1 - 9/30/2020, Task 1 - Steel Workstream',
  'Amount (Credit)': '',
  'Amount (Debit)': '2025.00',
  'Client': '',
  'Transaction Number': 'VENDBILL12387',
  'Type': 'Bill',
  'Vendor Rate': '',
  'Vendor Contract Expiration Date': '',
  'Nov 2020': 2025.0,
  'Total': 2025.0},
 {'Date': '2020-11-01T00:00:00',
  'Period': 'Nov 2020',
  'GL Account': '7210 Consultants and Contractors : Consultant Fees',
  'Document Number': '2020-004',
  'Program': 'Climate Aligned

In [10]:
sorts = ['Initiative', 'Project', 'Grant', 'GL Account']
sorts.reverse()

for p in programs:
    for s in sorts:
        programs[p].sort(key=lambda x: x[s])  
    

In [11]:
titles = """Initiative	Project	Grant	GL Account	Memo	Transaction Number	Vendor Name	Vendor Rate	Vendor Contract Expiration Date	Jul 2020	Aug 2020	Sep 2020	Oct 2020	Nov 2020	Dec 2020	Jan 2021	Feb 2021	Mar 2021	Apr 2021	May 2021	Jun 2021	Total
"""

titles = [y.strip("\n") for y in titles.split('\t')]

In [12]:
testpath = r'C:\Users\Jeremy Wendt\anaconda3\envs\Code\Detail_Trans_Project_FY21\reports\Tests\\'

In [13]:
months='Jul 2020	Aug 2020	Sep 2020	Oct 2020	Nov 2020	Dec 2020	Jan 2021	Feb 2021	Mar 2021	Apr 2021	May 2021	Jun 2021'

sumfields = set([y.strip("\n") for y in months.split('\t')])

In [14]:
def add_totals(data):
#if there is no data, return empty set
    if not len(data):
        return data
#get the initiative in the very first row
    ini = data[0].get('Initiative')
#have to make a new dataset of output, and append all the data into this new set along with subtotal and grand total
    output = []
    subtotal = 0.0
    monthly_subtotal = {}
    monthly_total = {}
    total = 0.0
    for d in data:
        total += float(d.get('Amount', 0.0))
        if d.get('Initiative') != ini:
            row = {'Vendor Contract Expiration Date': 'Subtotal', 'Total': subtotal}
            row.update(monthly_subtotal)
            output.append(row)
            subtotal = 0.0
            monthly_subtotal = {}
            ini = d.get('Initiative')
        output.append(d)
        subtotal += float(d.get('Amount', 0.0))
        for k in d:
#if the key is in sumfields and there is a value
            if k in sumfields and d.get(k) is not None:
                month = monthly_subtotal.get(k,0.0)
                month += float(d.get('Amount', 0.0))
                monthly_subtotal[k] = month
                month = monthly_total.get(k,0.0)
                month += float(d.get('Amount', 0.0))
                monthly_total[k] = month
    row = {'Vendor Contract Expiration Date': 'Subtotal', 'Total': subtotal}
    row.update(monthly_subtotal)
    output.append(row)    
    row = {'Vendor Contract Expiration Date': 'Grand Total', 'Total': total}    
    row.update(monthly_total)
    output.append(row)
    return output
            

In [15]:
def create_report(data, program):
    book = openpyxl.Workbook()
    sheet = book.active
    rows = add_totals(data[program])
    for i, t in enumerate(titles):
        sheet.cell(row = 1, column = i + 1).value = t 
    for i, r in enumerate(rows):
        for j, t in enumerate(titles):
            sheet.cell(row = i+2, column = j + 1).value = r.get(t,"")
    book.save(testpath + program + '.xlsx')

In [16]:
create_report(programs, "Climate Aligned Industries")

In [17]:
#for k in programs.keys():
#    create_report(programs, k)